In [ ]:
# imports
import cv2
import os
import random
import time
import numpy as np
import math

In [ ]:
def rotate_bound(image, angle):
    # grab the dimensions of the image and then determine the
    # center
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)

    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))

    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY

    # perform the actual rotation and return the image
    rotated_mat = cv2.warpAffine(image, M, (nW, nH))
    
    tmp = cv2.cvtColor(rotated_mat, cv2.COLOR_RGB2GRAY)
    _,alpha = cv2.threshold(tmp,0,255,cv2.THRESH_BINARY)
    b, g, r = cv2.split(rotated_mat)
    rgba = [b,g,r,alpha]
    dst = cv2.merge(rgba,4)
    
    return dst


def add_gaussian_noise(X_imgs):
    gaussian_noise_imgs = []
    row, col, _ = X_imgs[0].shape
    # Gaussian distribution parameters
    mean = 0
    var = 0.1
    sigma = var ** 0.5
    
    for X_img in X_imgs:
        gaussian = np.random.random((row, col, 1)).astype(np.float32)
        gaussian = np.concatenate((gaussian, gaussian, gaussian), axis = 2)
        gaussian_img = cv2.addWeighted(X_img, 0.75, 0.25 * gaussian, 0.25, 0)
        gaussian_noise_imgs.append(gaussian_img)
    gaussian_noise_imgs = np.array(gaussian_noise_imgs, dtype = np.float32)
    return gaussian_noise_imgs

def noise(img):
    mean = 0
    var = 10
    sigma = var ** 0.5
    gaussian = np.random.normal(mean, sigma, (100, 100)) #  np.zeros((224, 224), np.float32)
    noisy_image = np.zeros(img.shape, np.float32)
    

    if len(img.shape) == 2:
        noisy_image = img + gaussian
    else:
        noisy_image[:, :, 0] = img[:, :, 0] + gaussian
        noisy_image[:, :, 1] = img[:, :, 1] + gaussian
        noisy_image[:, :, 2] = img[:, :, 2] + gaussian

    cv2.normalize(noisy_image, noisy_image, 0, 255, cv2.NORM_MINMAX, dtype=-1)
    noisy_image = noisy_image.astype(np.uint8)
    return noisy_image


def increase_brightness(img, value=30):
    """ ??? %$&* """
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lim = 255 - (value if value > 0 else 0)
    v[v > lim] = 255
    #v[v < lim] = 0
    #v[v <= lim] += value
    v[v >= lim] -= value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img


def imageOverlay(background, overlay, x, y, width):
    """ overlay an image and set the width of the overlay """
    
    dim = (width, width)
    # resize image
    overlay = cv2.resize(overlay, dim, interpolation = cv2.INTER_AREA)

    background_width = background.shape[1]
    background_height = background.shape[0]

    if x >= background_width or y >= background_height:
        return background

    h, w = overlay.shape[0], overlay.shape[1]

    if x + w > background_width:
        w = background_width - x
        overlay = overlay[:, :w]

    if y + h > background_height:
        h = background_height - y
        overlay = overlay[:h]

    if overlay.shape[2] < 4:
        overlay = np.concatenate(
            [
                overlay,
                np.ones((overlay.shape[0], overlay.shape[1], 1), dtype = overlay.dtype) * 255
            ],
            axis = 2,
        )

    overlay_image = overlay[..., :3]
    mask = overlay[..., 3:] / 255.0

    background[y:y+h, x:x+w] = (1.0 - mask) * background[y:y+h, x:x+w] + mask * overlay_image

    return background

In [ ]:
overlay = cv2.imread('target_100.jpg')

for img in os.listdir("raw_dataset"):
    background = cv2.imread("raw_dataset/"+img)
    height, width, channels = background.shape
    overlay1 = noise(overlay)
    overlay2 = rotate_bound(overlay1, random.randint(0,360))
    new = imageOverlay(background, overlay2, x=random.randint(200,width-200), y=random.randint(200,height-200), width=150)
    cv2.imwrite(f"sim_dataset/{img}",new)


In [ ]:
img = cv2.imread('target_100.jpg')
mean = 0
var = 10
sigma = var ** 0.5
gaussian = np.random.normal(mean, sigma, (388, 394)) #  np.zeros((224, 224), np.float32)

noisy_image = np.zeros(img.shape, np.float32)

if len(img.shape) == 2:
    noisy_image = img + gaussian
else:
    noisy_image[:, :, 0] = img[:, :, 0] + gaussian
    noisy_image[:, :, 1] = img[:, :, 1] + gaussian
    noisy_image[:, :, 2] = img[:, :, 2] + gaussian

cv2.normalize(noisy_image, noisy_image, 0, 255, cv2.NORM_MINMAX, dtype=-1)
noisy_image = noisy_image.astype(np.uint8)

cv2.imshow("img", img)
cv2.imshow("gaussian", gaussian)
cv2.imshow("noisy", noisy_image)

cv2.waitKey(0)

In [ ]:
img = cv2.imread('target.jpg')

In [ ]:
outImg = cv2.imread("temp.jpg", 1)
tmp = cv2.cvtColor(outImg, cv2.COLOR_RGB2GRAY)
_,alpha = cv2.threshold(tmp,0,255,cv2.THRESH_BINARY)
b, g, r = cv2.split(outImg)
rgba = [b,g,r,alpha]
dst = cv2.merge(rgba,4)

In [ ]:
l =cv2.threshold(outImg,0,255,cv2.THRESH_BINARY)

In [ ]:
for img in os.listdir("dataset"):
    background = cv2.imread("dataset/"+img)
    img = increase_brightness(background, value=60)
    #background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
    #background = cv2.GaussianBlur(background, (5, 5), 0)
    #img_thresh = cv2.adaptiveThreshold(background,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,199,-50)
    utils.display(img)

In [ ]:
img = cv2.imread('target.jpg')
overlay = cv2.resize(img, (100,100), interpolation = cv2.INTER_AREA)
utils.display(overlay)
cv2.imwrite("targer_100.jpg", overlay)

In [ ]:
""" 
BLUR DETECTION 
https://www.pyimagesearch.com/2015/09/07/blur-detection-with-opencv/
"""

for img in os.listdir("dataset"):
    image = cv2.imread("dataset/"+img)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    fm = cv2.Laplacian(gray, cv2.CV_64F).var()
    text = "Not Blurry"
    # if the focus measure is less than the supplied threshold,
    # then the image should be considered "blurry"
    if fm < 100:
        text = "Blurry"
    # show the image
    cv2.putText(image, "{}: {:.2f}".format(text, fm), (100, 100),
        cv2.FONT_HERSHEY_SIMPLEX, 5, (0, 0, 255), 3)
    utils.display(image)